<a href="https://colab.research.google.com/github/edgasanc85/AzureOpenAI/blob/main/ChatGPT_El_Camino_a_Cristo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Tiene formato de código
```

# ChatBot basado en el libro EL CAMINO A CRISTO


## Instalar librerías principales y configuración de API Key de OpenAI

In [ ]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

Enter the secret value: ··········


## Carga de documents

In [ ]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://www.adven7.org/books/pdf/el_camino_a_cristo.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Contenido de ml_papers:



In [ ]:
type(ml_papers), len(ml_papers), ml_papers[6]

(list,
 144,
 Document(page_content='7\nque está en el seno del Padre, él le ha dado a conocer.”  (San\nJuan 1: 18). “Ni al Padre conoce nadie, sino el Hijo, y\naquel a quien el Hijo lo quisiere revelar.” (San Mateo 11:\n27). Cuando uno de sus discípulos le dijo: “Muéstranos al\nPadre”, Jesús respondió: “Tanto tiempo hace que estoy con\nvosotros, ¿y todavía no me conoces, Felipe? El que me ha\nvisto a mí, ha visto al Padre: ¿Cómo pues dices tú:\nMuéstranos al Padre?” (San Juan 14: 8, 9).\nJesús dijo, describiendo su misión terrenal: “Jehová me\nha ungido para anunciar buenas nuevas a los pobres; me a\nenviado para proclamar a los cautivos, y a los ciegos recobro\nde la vista, para poner en libertad a los oprimidos...” (San\nLucas 4: 18), esta era su obra. Pasó su vida haciendo bien y\nsanando a todos los oprimidos de Satanás.\nHabía aldeas enteras donde no se oía un gemido de\ndolor en casa alguna, porque él había pasado por ellas y\nsanado a todos sus enfermos. Su obra demostraba su d

## Split de documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200, #caracteres que se repiten en el fragmento pasado y siguiente para que tenga continuidad
    length_function= len
    )

documents = text_splitter.split_documents(ml_papers)

In [ ]:
len(documents), documents[10]

(278,
 Document(page_content='7\nque está en el seno del Padre, él le ha dado a conocer.”  (San\nJuan 1: 18). “Ni al Padre conoce nadie, sino el Hijo, y\naquel a quien el Hijo lo quisiere revelar.” (San Mateo 11:\n27). Cuando uno de sus discípulos le dijo: “Muéstranos al\nPadre”, Jesús respondió: “Tanto tiempo hace que estoy con\nvosotros, ¿y todavía no me conoces, Felipe? El que me ha\nvisto a mí, ha visto al Padre: ¿Cómo pues dices tú:\nMuéstranos al Padre?” (San Juan 14: 8, 9).\nJesús dijo, describiendo su misión terrenal: “Jehová me\nha ungido para anunciar buenas nuevas a los pobres; me a\nenviado para proclamar a los cautivos, y a los ciegos recobro\nde la vista, para poner en libertad a los oprimidos...” (San\nLucas 4: 18), esta era su obra. Pasó su vida haciendo bien y\nsanando a todos los oprimidos de Satanás.\nHabía aldeas enteras donde no se oía un gemido de\ndolor en casa alguna, porque él había pasado por ellas y\nsanado a todos sus enfermos. Su obra demostraba su divina\n

## Embeddings e ingesta a base de datos vectorial

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs = {"k": 3}
    )

## Modelos de chat y cadenas para consulta de información

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name= "gpt-3.5-turbo",
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
query = "cual es la mayor necesidad del hombre?"
qa_chain.run(query)

KeyboardInterrupt: ignored

In [ ]:
query = "por qué debo nacer de nuevo?"
qa_chain.run(query)

'De acuerdo con el texto, se menciona que es necesario nacer de nuevo para poder ver el reino de Dios. Además, se dice que el hombre natural no puede recibir las cosas del Espíritu de Dios y que solo a través de un nuevo nacimiento espiritual se puede participar de la vida que Cristo vino a dar. En resumen, el nuevo nacimiento es necesario para tener una nueva vida en Cristo y ser parte del reino de Dios.'

In [ ]:
query = "cómo profundizar nuestro caminar con Dios?"
qa_chain.run(query)

'Para profundizar nuestro caminar con Dios, es importante tener una relación íntima y constante con Él. Aquí hay algunas sugerencias:\n\n1. Oración: Dedica tiempo diario para hablar con Dios, expresarle tus pensamientos, preocupaciones y agradecimientos. La oración es una forma de comunicación directa con Dios y fortalece nuestra conexión con Él.\n\n2. Estudio de la Biblia: Lee y medita en la Palabra de Dios regularmente. La Biblia es la revelación de Dios y nos enseña sobre su carácter, sus promesas y su voluntad para nuestras vidas. A medida que profundizamos en su Palabra, crecemos en nuestro conocimiento y comprensión de Dios.\n\n3. Comunión con otros creyentes: Busca la compañía de otros cristianos que compartan tu fe. Participa en grupos de estudio bíblico, iglesias o comunidades de creyentes donde puedas aprender y crecer juntos en la fe.\n\n4. Servicio y obediencia: Busca oportunidades para servir a Dios y a los demás. La obediencia a los mandamientos de Dios y el servicio desi

In [ ]:
query = "como orar para que las oraciones sean contestadas?"
qa_chain.run(query)

'Según el texto proporcionado, hay algunas condiciones que debemos cumplir para que nuestras oraciones sean contestadas:\n\n1. Sentir necesidad de la ayuda de Dios: Debemos reconocer nuestra dependencia de Dios y anhelar su intervención en nuestras vidas.\n\n2. Confesar y arrepentirse de nuestros pecados conocidos: Debemos examinar nuestro corazón y confesar cualquier pecado conocido a Dios, buscando su perdón y limpieza.\n\n3. Perdonar a los demás: Debemos perdonar a aquellos que nos han ofendido, ya que no podemos esperar que nuestras oraciones sean contestadas si tenemos un espíritu de falta de perdón.\n\n4. Perseverar en la oración: Debemos ser persistentes y constantes en nuestra vida de oración, buscando a Dios en todo momento y en todas las circunstancias.\n\n5. Mantener una comunión continua con Dios: Debemos esforzarnos por mantener una conexión constante con Dios, buscando una unión ininterrumpida con Él y permitiendo que su vida fluya en nosotros.\n\nEn resumen, para que nue

In [ ]:
query = "Como se hara justo un pecador?"
qa_chain.run(query)

'Un pecador puede ser justificado a través de la fe en Jesucristo. Según el contexto proporcionado, Cristo ofrece quitarnos nuestros pecados y vestirnos de su justicia. Si nos entregamos a él y lo aceptamos como nuestro Salvador, seremos contados entre los justos por consideración a él. El carácter de Cristo toma el lugar del nuestro y somos aceptados por Dios como si no hubiéramos pecado. Además, Cristo cambia el corazón y habita en nosotros por la fe. Debemos mantener esta comunión con Cristo a través de la fe y la sumisión continua de nuestra voluntad a él. En resumen, la justificación de un pecador se logra a través de la fe en Jesucristo y su obra redentora.'